# External Integration — Report

This notebook visualizes the CSV outputs generated by the **External_Integration** block.

## What this notebook shows
- **External SDKs**: Group → Artifact treemap, Top artifacts (Bar), and Group-level summary (Bar).
- **Hardcoded URLs**: Top hosts (Bar), Host → Class treemap, HTTP vs HTTPS share (Donut), and a compact table.

> If a CSV is missing or empty, the cell prints an info message and skips the chart.


In [ ]:
# Setup: imports, paths, helpers
# - CSVs are read from reports/csv-reports/<CATEGORY>/<file>.csv relative to this notebook folder.
# - Minimal console output; only show information if a CSV is missing/empty.
# - Bar charts use an explicit default color so it's easy to tweak later.
# - Titles are standardized without block prefixes.

import os
from pathlib import Path
from urllib.parse import urlparse
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('future.no_silent_downcasting', True)

CATEGORY = "External_Integration"
CSV_BASE = Path("../reports/csv-reports").resolve()
EXT_DIR = CSV_BASE / CATEGORY

# Explicit default color for all bar charts in this notebook
DEFAULT_BAR_COLOR = ["#1f77b4"]

# CSV IO helpers
NA_LITS = ["", " ", "NA", "N/A", "n/a", "NaN", "NULL", "Null", "null", "None", "none", "-", "--"]

def read_csv_safe(p: Path) -> pd.DataFrame:
    """Read a CSV if present; otherwise return an empty DataFrame.
    Prints a minimal info message when missing or unreadable."""
    p = Path(p)
    if not p.exists():
        print(f"[info] Missing CSV: {p}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(p, na_values=NA_LITS, keep_default_na=True)
        df.columns = [str(c).strip() for c in df.columns]
        df = df.dropna(how="all")
        return df
    except Exception as e:
        print(f"[warn] Failed to read {p}: {e}")
        return pd.DataFrame()

def labelize_na(s, label="N/A"):
    s = s.copy()
    s = s.mask(s.isna(), label).astype(str)
    s = s.replace({"nan": label, "NaN": label})
    return s

def find_col(df, *cands, default=None, contains=None):
    """Find a column by exact candidates or by substring (contains)."""
    low = {c.lower(): c for c in df.columns}
    for c in cands:
        if c and c.lower() in low:
            return low[c.lower()]
    if contains:
        for k, orig in low.items():
            if contains.lower() in k:
                return orig
    return default

# Import shared chart functions from interface
import sys
sys.path.append(str(Path("../interface").resolve()))
from charts.external_integration_charts import (
    create_sdk_treemap,
    create_top_artifacts_bar,
    create_groups_bar,
    create_top_hosts_bar,
    create_host_class_treemap,
    create_scheme_share_donut
)


## 1) External SDKs — usage overview

In [ ]:
# Charts for External_SDKs
# Where charts are generated:
#  - 1A) External SDK usage (Group → Artifact) (Treemap)
#  - 1B) Top external SDK artifacts by usage (Bar, explicit color)
#  - 1C) External SDK usage by group (Bar, explicit color)

path = EXT_DIR / "External_SDKs.csv"
df_sdks = read_csv_safe(path)

if not df_sdks.empty:
    c_grp = find_col(df_sdks, "artifactGroup", "group", contains="group", default=None)
    c_art = find_col(df_sdks, "artifactName", "name", contains="name", default=None)

    if c_grp and c_art:
        # 1A) Treemap — Group → Artifact sized by count
        fig = create_sdk_treemap(df_sdks, c_grp, c_art)
        if fig:
            fig.show()

        # 1B) Top artifacts by usage (bar)
        fig2 = create_top_artifacts_bar(df_sdks, c_grp, c_art)
        if fig2:
            fig2.show()

        # 1C) Group-level summary (bar)
        fig3 = create_groups_bar(df_sdks, c_grp, c_art)
        if fig3:
            fig3.show()
    else:
        print("[info] External_SDKs.csv lacks expected columns — skipping charts.")
else:
    print("[info] External_SDKs.csv missing or empty.")


## 2) Hardcoded URLs — endpoints and hosts

In [ ]:
# Charts for Hardcoded_URLs
# Where charts are generated:
#  - 2A) Top hardcoded URL hosts (Bar, explicit color)
#  - 2B) Hardcoded URLs — Host → Declaring Class (Treemap; limited to top hosts)
#  - 2C) Scheme share (http vs https) (Donut)
#  - 2D) Compact table (first rows)

path = EXT_DIR / "Hardcoded_URLs.csv"
df_urls = read_csv_safe(path)

if not df_urls.empty:
    c_ep  = find_col(df_urls, "endpoint", contains="endpoint", default=None)
    c_cls = find_col(df_urls, "declaringClass", contains="class", default=None)
    c_fld = find_col(df_urls, "fieldName", contains="field", default=None)

    if c_ep:
        # 2A) Top hosts (bar)
        fig = create_top_hosts_bar(df_urls, c_ep)
        if fig:
            fig.show()

        # 2B) Host → Class treemap (limited to top hosts)
        if c_cls:
            fig2 = create_host_class_treemap(df_urls, c_ep, c_cls)
            if fig2:
                fig2.show()

        # 2C) HTTP vs HTTPS share (donut)
        fig3 = create_scheme_share_donut(df_urls, c_ep)
        if fig3:
            fig3.show()

        # 2D) Compact table (first rows)
        from IPython.display import display
        cols = [c for c in [c_ep, c_cls, c_fld] if c]
        display(df_urls[cols].head(100))
    else:
        print("[info] Hardcoded_URLs.csv lacks expected columns — skipping charts.")
else:
    print("[info] Hardcoded_URLs.csv missing or empty.")
